to do:
- image in tensorboard (test alpha mobilenet)
- imports aufräumen
- Secondstage.py & second_stage_utils.py importieren
- ss_utils -> Bild öffnen
- Konfiguration außerhalb:
    - Name output
    - cat/ bin/ reg
- endliche Anzahl an Runden

In [0]:
%tensorflow_version 1.x

von Tobi geklaut


In [0]:
!git clone --quiet https://github.com/tensorflow/models.git

In [0]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

In [4]:
%cd models/research

/content/models/research


In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

In [0]:
!python object_detection/builders/model_builder_test.py

In [8]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/SecondStage_colab/')

In [0]:
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/SecondStage_colab/'

Second Stage

In [0]:
#NAME =
#example: cat_exptype_dataset(training)_dataset(eval)_conf(batch size etc) 

In [0]:
LOG_PATH = '/content/gdrive/My Drive/SecondStage_colab/output/'
LABEL_MAP_PATH = '/content/gdrive/My Drive/SecondStage_colab/label_map.pbtxt'
TRAIN_RECORD = '/content/gdrive/My Drive/data/train/training_rot15_32400.tfrecords'
EVAL_RECORD = '/content/gdrive/My Drive/data/test/sphero/sphero_evaluation_rot2_4320_.tfrecords'

In [12]:
os.chdir('/content/gdrive/My Drive/')

from SecondStage_colab import second_stage_utils
from SecondStage_colab import file_utils

Using TensorFlow backend.


In [0]:
os.chdir('/content/models/research')
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

In [0]:
#os.environ['PYTHONPATH'] += ':/contentdrive/My Drive/SecondStage_colab/'

configuration

In [0]:
GPU = True
BATCH_SIZE = 1024

ANGLES_PER_BIN = 1
NUM_ORI_BINS = 360

label map


In [0]:
os.chdir('/content/gdrive/My Drive/SecondStage_colab')

In [17]:
%%writefile label_map.pbtxt

item {
  id: 1
  name: 'bright_blue'
}

item {
  id: 2
  name: 'bright_red'
}

item {
  id: 3
  name: 'bright_green'
}

item {
  id: 4
  name: 'bright_white'
}

item {
  id: 5
  name: 'dark_blue'
}

item {
  id: 6
  name: 'dark_green'
}

item {
  id: 7
  name: 'dark_red'
}

Overwriting label_map.pbtxt


experiment definitioin


In [18]:
%%writefile exp_def.py

from copy import deepcopy

default_sstage_conf = {
    'dataset': 'default',
    'types': ['copter','sphero','youbot'],
    # converged after 15 / 10 / 5 epochs
    # 10 epochs after convergance for evaluation phase
    'epochs_cat': 25,
    #'epochs_reg': 20,
    'epochs_reg': 50,
    'epochs_bin': 15,
    'optimizer': 'adam',
    'learning_rate': 3e-4,
    'dropout': 0,
    'alpha': 0.5,
    'img_size': 35,
    'separate_cat_ori': True,
    'cat_weight': 1.0,
    'reg_weight': 1.0,
    'bin_weight': 1.0,
    'enable_reg': False,
    'enable_bin': True,
    'repetions': 4,
}

def create_all_sstage_experiments():
    configs = []
    configs.extend(create_sstage_default())

    return configs

def create_sstage_default():
    config = []
    modified = deepcopy(default_sstage_conf)
    modified['name'] = "sstage_default"
    modified['enable_reg'] = True
    config.append(deepcopy(modified))
    return config

Overwriting exp_def.py


In [0]:
from tqdm import tqdm
import re
from datetime import datetime
from multiprocessing import Process, Queue

from copy import deepcopy

from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Reshape, Conv2D, Activation
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.models import load_model
from keras.utils import np_utils

import numpy as np
from numpy.random import randint

import tensorflow as tf
from PIL import Image
import io

import csv, json, pickle
from lxml import etree


In [0]:
def make_square(im, size, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    scl = size/max(x, y)
    im = im.resize((int(x*scl),int(y*scl)))
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - im.size[0]) / 2), int((size - im.size[1]) / 2)))
    return new_im

def angle_diff2(y_true, y_pred):
    return tf.mod(( (y_true - y_pred) + 180 ), 360 ) - 180

def np_angle_diff2(y_true, y_pred):
    return np.mod(( (y_true - y_pred) + 180 ), 360 ) - 180

def angle_mae(y_true, y_pred):
    return K.mean(K.abs(angle_diff2(y_true, y_pred)), axis=-1)

def angle_mse(y_true, y_pred):
    return K.mean(K.square(angle_diff2(y_true, y_pred)), axis=-1)

def angle_bin_error(y_true, y_pred):
    diff = angle_diff2(K.argmax(y_true)*ANGLES_PER_BIN,
                       K.argmax(y_pred)*ANGLES_PER_BIN)
    return K.mean(K.cast(K.abs(diff), K.floatx()))

def data_to_keras(X,Y,Z, num_classes, size=35):
    X = [np.asarray(e) for e in X]

    X = np.asarray(X, dtype="uint8")
    X = X.reshape(X.shape[0], size, size, 3)
    X.astype('float32')
    #X /= 255
    Y = np.asarray(Y)
    Y = np_utils.to_categorical(Y, num_classes)
    Z = np.asarray(Z)
    return X,Y,Z

def angle_to_bin(Z):
    Z = np.mod(Z, 360)
    return np_utils.to_categorical(np.floor(Z/ANGLES_PER_BIN),NUM_ORI_BINS)

"""
img_enc = (feats.feature['image/encoded'].bytes_list.value[0]).decode('utf-8')
img_name = (feats.feature['image/filename'].bytes_list.value[0]).decode('utf-8')
filename = (feats.feature['image/filename'].bytes_list.value[0]).decode('utf-8')
source_id = (feats.feature['image/source_id'].bytes_list.value[0]).decode('utf-8')
width = feats.feature['image/width'].int64_list.value[0]
height = feats.feature['image/height'].int64_list.value[0]
class = (feats.feature['image/object/class/text'].bytes_list.value[0]).decode('utf-8')
class_label = feats.feature['image/object/class/label'].int64_list.value[0]
color = (feats.feature['image/object/subclass/text'].bytes_list.value[0]).decode('utf-8')
color_id = feats.feature['image/object/subclass/label'].int64_list.value[0]
orientation = feats.feature['image/object/pose/orientation'].int64_list.value[0]
"""

def tf_record_load_crops(files,num_per_record=-1,size=35):
    crops, classes, orientations = [],[],[]
    debug_infos = []
    for f in files:
        record_iterator = tf.python_io.tf_record_iterator(f)
        #records = tf.data.TFRecordDataset(f)
        for l, string_record in enumerate(record_iterator):
        #for string_record in records:
            if num_per_record!=-1 and l > num_per_record: break
            example = tf.train.Example()
            example.ParseFromString(string_record)#.numpy())
            feats = example.features

            img_enc = (feats.feature['image/encoded'].bytes_list.value[0])#.decode('utf-8')

            img_name = (feats.feature['image/filename'].bytes_list.value[0]).decode('utf-8')
            img_enc = (feats.feature['image/encoded'].bytes_list.value[0]) #.decode('utf-8')
            width = feats.feature['image/width'].int64_list.value[0]
            height = feats.feature['image/height'].int64_list.value[0]

            img = Image.frombytes('RGB', (height, width), img_enc)
            #img = Image.open(io.BytesIO(img_enc))

            color = (feats.feature["image/object/subclass/text"].bytes_list.value[0]).decode('utf-8') 
            color_id = feats.feature["image/object/subclass/label"].int64_list.value[0]
            orientation = feats.feature["image/object/pose/orientation"].int64_list.value[0]

            crops.append(make_square(img,size))
            classes.append(color_id)
            orientations.append(orientation)
            debug_infos.append({
                'filename': img_name,
                'src_record': f
            })

    assert len(crops) == len(classes) and len(crops) == len(orientations)
    print("Loaded {} crops from {}".format(len(crops),files))

    return crops, classes, orientations, debug_infos

def custom_randint(min, max):
    min = round(min)
    max = round(max)
    if min == max:
        return min
    return randint(min, max)

def tf_record_extract_crops(files, num_derivations,
                            out_var, in_var,
                            num_per_record=-1,
                            size=35,
                            class_filters=None):
    crops, classes, orientations = [],[],[]
    debug_infos = []
    for f in files:
        record_iterator = tf.python_io.tf_record_iterator(f)
        for l, string_record in enumerate(record_iterator):
            if num_per_record!=-1 and l > num_per_record: break
            example = tf.train.Example()
            example.ParseFromString(string_record) #.numpy())
            feats = example.features
            width  = feats.feature["image/width"].int64_list.value[0]
            height = feats.feature["image/height"].int64_list.value[0]
            #width = 35
            #height = 35
            img_name = (feats.feature['image/filename'].bytes_list.value[0]).decode('utf8')
            img_enc = (feats.feature['image/encoded'].bytes_list.value[0])
            #img = Image.open(io.BytesIO(img_enc))
            #img = Image.open(open(img_enc, "rb"))
            img = Image.frombytes('RGB', (height, width), img_enc)

            for i,_ in enumerate(feats.feature["image/object/class/text"].bytes_list.value):
                class_text = (feats.feature["image/object/subclass/text"].bytes_list.value[i]).decode('utf8')
                if not (class_filters == None or any(m in class_text for m in class_filters)):
                    continue
                class_label = feats.feature["image/object/subclass/label"].int64_list.value[i]
                orientation = feats.feature["image/object/pose/orientation"].int64_list.value[i]
                width = feats.feature['image/width'].int64_list.value[0]
                height = feats.feature['image/height'].int64_list.value[0]                
 
                img_resized = img;
                if (width != 35 or height != 35): 
                    img_resized = img.resize((35,35), Image.BICUBIC)

                crops.append(make_square(img_resized,size))
                classes.append(class_label)
                orientations.append(orientation)
                debug_infos.append({
                    'filename': img_name,
                    'src_record': f,
                    'crop_num': i*num_derivations,
                })
            img.close()

    return crops, classes, orientations, debug_infos

from keras.callbacks import Callback
import sklearn.metrics as sklm
class TensorBoardCustom(Callback):
    def __init__(self, log_dir='./logs',label_map='',images=''):
        super(Callback, self).__init__()
        global tf, projector
        try:
            import tensorflow as tf
            from tensorflow.contrib.tensorboard.plugins import projector
        except ImportError:
            raise ImportError('You need the TensorFlow module installed to use TensorBoard.')

        self.log_dir = log_dir
        self.label_map = label_map
        self.images = images

    def set_model(self, model):
        self.model = model
        if K.backend() == 'tensorflow':
            self.sess = K.get_session()
        self.merged = tf.summary.merge_all()
        self.writer = tf.summary.FileWriter(self.log_dir)

    def write_metric(self, name, value, epoch, namespace=''):
        if namespace != '':
            namespace += '/'
        summary = tf.Summary(value=[
            tf.Summary.Value(tag=namespace+name, simple_value=value),
        ])
        self.writer.add_summary(summary, epoch)

    def add_custom_metrics(self, epoch):
        y_pred, z_pred, z2_pred = self.model.predict(self.validation_data[0])
        # [1] -> Y; [2] -> Z
        y_targ = self.validation_data[1]
        y_targ_onehot = np.argmax(y_targ, axis=1)
        y_pred_onehot = np.argmax(y_pred, axis=1)

        class_prec, class_rec, class_f1, class_support = sklm.precision_recall_fscore_support(
            y_targ_onehot,
            y_pred_onehot,
            labels=[int(e) for e in self.label_map.keys()]
        )
        for i, (prec, rec, f1, sup) in enumerate(zip(class_prec,
                                                   class_rec,
                                                   class_f1,
                                                   class_support)):
            i+=1
            if self.label_map == '':
                label = str(i)
            elif i in self.label_map:
                label = str(i) +' '+ self.label_map[i]['name']
            else:
                label = str(i)
            if sup > 0:
                namespace = "Precision by Category/{}".format(label)
                self.write_metric('Precision', prec, epoch, namespace)
                namespace = "Recall by Category/{}".format(label)
                self.write_metric('Recall', rec, epoch, namespace)
                namespace = "F1 by Category/{}".format(label)
                self.write_metric('F1', f1, epoch, namespace)
            if epoch == 0:
                namespace = "Support by Category/{}".format(label)
                self.write_metric('Support', sup, epoch, namespace)
        # average='weighted'
        # Calculate metrics for each label, and find their average
        # -> balanced
        avg_prec, avg_rec, avg_f1, avg_support = sklm.precision_recall_fscore_support(
            y_targ_onehot,
            y_pred_onehot,
            average='weighted'
        )
        self.write_metric('Average Precision', avg_prec, epoch, 'Average Performance')
        self.write_metric('Average Recall', avg_rec, epoch, 'Average Performance')
        self.write_metric('Average F1', avg_f1, epoch, 'Average Performance')
        self.write_metric('Average Support', avg_support, epoch, 'Average Performance')

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for name, value in logs.items():
            if name in ['batch', 'size']:
                continue
            # summary = tf.Summary()
            # summary_value = summary.value.add()
            # summary_value.simple_value = value.item()
            # summary_value.tag = name
            if 'val' in name:
                group = 'Keras Validation/'
            else:
                group = 'Keras Training/'
            self.write_metric(name, value, epoch, group)

        # self.add_custom_metrics(epoch)

        self.writer.flush()

    def on_train_end(self, _):
        self.writer.close()


import unittest

class TestSecondStageUtils(unittest.TestCase):

    def test_np_angle_diff2(self):
        self.assertEqual(np_angle_diff2(359,52), -53)
        self.assertEqual(np_angle_diff2(52,359), 53)
        self.assertEqual(np_angle_diff2(0,720), 0)

    def test_angle_to_bin(self):
        self.assertEqual(len(angle_to_bin(4)), 90)
        self.assertEqual(angle_to_bin(4)[0], 0)
        self.assertEqual(angle_to_bin(4)[1], 1)
        for i in range(2,90):
            self.assertEqual(angle_to_bin(4)[i], 0)
        self.assertEqual(angle_to_bin(50)[12], 1)
        self.assertEqual(angle_to_bin(360)[0], 1)
        self.assertEqual(angle_to_bin(370)[2], 1)
        self.assertEqual(angle_to_bin(-10)[87], 1)

#if __name__ == '__main__':
#    unittest.main()

In [0]:
def train(train_record, conf, types, out, rep=1):

    #from SecondStage_colab.file_utils import *
    from file_utils import save_json

    timestamp = "{:%Y-%m-%d-%H-%M}".format(datetime.now())
    log_path = LOG_PATH + conf['name'] + '_' + \
                ''.join(types) + out + '/' + \
                timestamp + '-r' + str(rep) + '/'
    os.makedirs(log_path, exist_ok=True)
    save_json(log_path + '/experiment_config.json', conf)

    label_map = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)
    #label_map = create_category_index_from_labelmap(LABEL_MAP_PATH)

    num_classes = label_map_util.get_max_label_map_index(
                           label_map_util.load_labelmap(LABEL_MAP_PATH)) + 1

    X,Y,Z,_ = tf_record_load_crops([train_record])
    X_train, Y_train, Z_train = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_train = angle_to_bin(Z_train)
    print("´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´")
    print(Y_train.shape)
    print(num_classes)
    print(Z2_train.shape)
    print(NUM_ORI_BINS)
    print("´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´")

    eval_records = EVAL_RECORD

    X,Y,Z,_ = tf_record_extract_crops([eval_records], 1, 0.0, 0.0)
    X_val, Y_val, Z_val = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_val = angle_to_bin(Z_val)

    assert len(X_val) > 0 and len(Y_val) > 0 and len(Z_val) > 0, \
        '{} is incomplete'.format(eval_records)
    mobilenet_base = applications.mobilenet.MobileNet(alpha = conf['alpha'],
                                                      weights = "imagenet",
                                                      include_top=False,
                                                      input_shape = (
                                                      conf['img_size'],
                                                      conf['img_size'],
                                                        3
                                                      ))
    shape = (1, 1, int(1024 * conf['alpha']))
    x = GlobalAveragePooling2D()(mobilenet_base.output)
    x = Reshape(shape, name='reshape_1')(x)
    x = Dropout(conf['dropout'], name='dropout')(x)
    # Branch regression
    reg = Conv2D(1, (1, 1),
               padding='same', name='conv_reg')(x)
    reg = Activation('linear', name='act_linear')(reg)
    reg = Reshape((1,), name='reg_out')(reg)
    # Branch orientation classification with bins
    bin = Conv2D(NUM_ORI_BINS, (1, 1),
               padding='same', name='conv_bin')(x)
    bin = Activation('softmax', name='act_bin')(bin)
    bin = Reshape((NUM_ORI_BINS,), name='bin_out')(bin)
    # Branch classification
    cat = Conv2D(num_classes, (1, 1),
               padding='same', name='conv_cat')(x)
    cat = Activation('softmax', name='act_softmax')(cat)
    cat = Reshape((num_classes,), name='cat_out')(cat)

    # creating the final model
    model_final = None
    model_final = Model(inputs = mobilenet_base.input, outputs = [cat,reg,bin])

    if conf['optimizer'] == 'rmsprop':
        optimizer = optimizers.RMSprop(lr=conf['learning_rate'])
    elif conf['optimizer'] == 'adam':
        optimizer = optimizers.Adam(lr=conf['learning_rate'])
    else:
        raise Error('Unknown optimizer: ' + conf['optimizer'])

    model_final.compile(optimizer = optimizer,
                        loss={'cat_out': 'categorical_crossentropy',
                              'reg_out': angle_mse,
                              'bin_out': 'categorical_crossentropy',
                        },
                        loss_weights={'cat_out': conf['cat_weight'],
                                      'reg_out': conf['reg_weight'],
                                      'bin_out': conf['bin_weight']},
                        metrics ={'cat_out': 'accuracy',
                                  'reg_out': angle_mae,
                                  'bin_out': angle_bin_error})

    ###########

    img = np.reshape(X_train[0], (-1, 35, 35, 1))
    !rm -rf logs
    logdir = LOG_PATH + datetime.now().strftime("%Y%m%d-%H%M%S")
    #file_writer = tf.summary.create_file_writer(logdir)
    file_writer = tf.compat.v1.summary.FileWriter(logdir)
    with file_writer: #.as_default():
      #tf.summary.image("Training data", img, step=0)
      tf.compat.v1.summary.image("Training data", img) #, step=0)


    ########

    summary = TensorBoardCustom(log_dir=log_path,label_map=label_map)
    filepath=log_path+"model-{epoch:02d}.h5"
    checkpoint = ModelCheckpoint(filepath, verbose=1, period=5)

    model_final.fit(
        X_train,
        {'cat_out': Y_train, 'reg_out': Z_train, 'bin_out': Z2_train},
        validation_data=(X_val,[Y_val,Z_val, Z2_val]),
        batch_size=BATCH_SIZE, epochs=conf['epochs'], verbose=0,
        callbacks=[summary,checkpoint],
        shuffle=True
    )
    model_final.save(log_path+"model-final.h5")
    print("Finished training for {}_{}".format(conf['name'],t))


In [0]:
from SecondStage_colab import exp_def
from copy import deepcopy

In [42]:
exp = exp_def.create_all_sstage_experiments()
for or_conf in tqdm(exp):
    print("or_conf", or_conf)
    for r in range(or_conf['repetions']): 
        #train_records = get_recursive_file_list(TRAIN_DIR,file_matchers=[or_conf['dataset']])
        train_records = [TRAIN_RECORD]
        for t in or_conf['types']:
            for out in ['_cat','_reg','_bin','']:
                conf = deepcopy(or_conf)
                train_instance_name = conf['name']+'_'+t+out
                if not conf['separate_cat_ori'] and out != '':
                    continue
                if conf['separate_cat_ori']:
                    if out == '':
                        continue
                    elif out == '_cat':
                        conf['reg_weight'] = 0.0
                        conf['bin_weight'] = 0.0
                        conf['epochs'] = conf['epochs_cat']
                    elif out == '_reg' and conf['enable_reg']:
                        conf['cat_weight'] = 0.0
                        conf['bin_weight'] = 0.0
                        conf['epochs'] = conf['epochs_reg']
                    elif out == '_bin' and conf['enable_bin']:
                        conf['cat_weight'] = 0.0
                        conf['reg_weight'] = 0.0
                        conf['epochs'] = conf['epochs_bin']
                    else:
                        print('UNKNOWN OUTPUT CONFIG {}'.format(out))
                        continue
                if not re.match('sstage_default_sphero_reg', train_instance_name): 
                    print('Skip '+train_instance_name)
                    continue            
                #record_for_type = [e for e in train_records if t in e]
                #print("record_for_type: ", len(record_for_type))
                #assert len(record_for_type) == 1, \
                #       "{} for {} has not one item".format(record_for_type, train_instance_name)
                print("Start training")
                
                if not GPU:
                    p = Process(
                      target=train,
                      args=(train_records[0], conf, ['sphero'], out, r)
                    )
                    p.start() 
                else:
                 
                  train(train_records[0], conf, ['sphero'], out)








  0%|          | 0/1 [00:00<?, ?it/s]

or_conf {'dataset': 'default', 'types': ['copter', 'sphero', 'youbot'], 'epochs_cat': 25, 'epochs_reg': 50, 'epochs_bin': 15, 'optimizer': 'adam', 'learning_rate': 0.0003, 'dropout': 0, 'alpha': 0.5, 'img_size': 35, 'separate_cat_ori': True, 'cat_weight': 1.0, 'reg_weight': 1.0, 'bin_weight': 1.0, 'enable_reg': True, 'enable_bin': True, 'repetions': 4, 'name': 'sstage_default'}
Skip sstage_default_copter_cat
Skip sstage_default_copter_reg
Skip sstage_default_copter_bin
Skip sstage_default_sphero_cat
Start training
Loaded 32400 crops from ['/content/gdrive/My Drive/data/train/training_rot15_32400.tfrecords']
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´
(32400, 8)
8
(32400, 360)
360
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


INFO:tensorflow:Summary name Training data is illegal; using Training_data instead.

Epoch 00005: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-13-r1/model-05.h5

Epoch 00010: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-13-r1/model-10.h5

Epoch 00015: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-13-r1/model-15.h5

Epoch 00020: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-13-r1/model-20.h5

Epoch 00025: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-13-r1/model-25.h5

Epoch 00030: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-13-r1/model-30.h5

Epoch 00035: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_d

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


INFO:tensorflow:Summary name Training data is illegal; using Training_data instead.

Epoch 00005: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-17-r1/model-05.h5

Epoch 00010: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-17-r1/model-10.h5

Epoch 00015: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-17-r1/model-15.h5

Epoch 00020: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-17-r1/model-20.h5

Epoch 00025: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-17-r1/model-25.h5

Epoch 00030: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_reg/2020-01-14-20-17-r1/model-30.h5

Epoch 00035: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_d

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


INFO:tensorflow:Summary name Training data is illegal; using Training_data instead.


KeyboardInterrupt: ignored